# Equity Premium Prediction Analysis

This notebook follows the (Rapach 2010) [^1] to implement the prediction performance analysis.

[^1]: Rapach, D. E., Strauss, J. K., & Zhou, G. (2010). Out-of-Sample Equity Premium Prediction: Combination Forecasts and Links to the Real Economy. The Review of Financial Studies, 23(2), 821–862. https://doi.org/10.1093/rfs/hhp063


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import sys
sys.path.append('../module')

from analysis import get_period_return
from analysis import get_oos_r_square,\
                     get_p_value_of_MSPE_adjusted_test,\
                     get_significance_of_MSPE_adjusted_test,\
                     get_utility_gain_from_prediction,\
                     get_significance_of_p_value
from data_handler import get_monthly_date_format
from data_handler import get_quarterly_date_format
from data_handler import get_econ_predictors
from IO_handler import post_dataframe_to_latex_table

%matplotlib inline

In (Rapach 2010), they adopt three measurements to evaluate the performance of the prediction of equity premium.

1. [$R^2_{OS}$: out of sample $R^2$](##out-of-sample-$r^2$)
2. [MSPE - adjusted statistic: The significance of the $R^2_{OS}$](##mspe-adjusted-test-(mean-squared-prediction-error))
3. [$\Delta$: The utility gain](##the-utility-gain)

load prediction results

In [3]:
START_DATE = '1965-01'
END_DATE = '2005-04'
prediction_df = pd.read_csv('../../data/ml_prediction_1947_2005.csv', index_col=0, parse_dates=True, date_parser=get_monthly_date_format)
prediction_df = prediction_df.loc[START_DATE:END_DATE]
equity_premium = prediction_df.pop('Equity Premium')
historical_average = prediction_df.pop('Historical Average')

## out of sample $R^2$

$$
\begin{equation}
R_{O S}^2=1-\frac{\sum_{k=q_0+1}^q\left(r_{m+k}-\hat{r}_{m+k}\right)^2}{\sum_{k=q_0+1}^q\left(r_{m+k}-\bar{r}_{m+k}\right)^2}
\end{equation}
$$

Where $m$ is the size of in sample data and $q_0$ is the origin of the hold out period.

In [3]:
def get_oos_r_square(y_hat: np.ndarray, y: np.ndarray, y_bar: np.ndarray) -> float:
    """
    This function calculates the out-of-sample R square for a prediction.

    Parameters
    ----------
    y_hat : np.ndarray
        Prediction values.
    y : np.ndarray
        True values.
    y_bar : np.ndarray
        Historical average.

    Returns
    -------
    float
        Out-of-sample R square.
    """
    ss_res = np.sum((y - y_hat) ** 2)
    ss_tot = np.sum((y - y_bar) ** 2)
    R_2 = 1 - ss_res / ss_tot
    R_2_percentage = R_2 * 100

    return R_2_percentage

In [24]:
R_2_OOS = prediction_df.apply(lambda x: get_oos_r_square(y=equity_premium, y_hat=x, y_bar=historical_average))
R_2_OOS

Elastic Net       0.078502
Random Forest   -10.215775
dtype: float64

## MSPE adjusted test (Mean Squared Prediction Error)

We test whether a prediction method is significantly different from the historical average. We follow the statistical test by Clark and West (2007). Rapach (2010) also uses this statistical test method.

The null hypothesis is H0: $R_{OS}^2 < 0$

The test statistics is:
\begin{equation}
f_{t+1}=\left(r_{t+1}-\bar{r}_{t+1}\right)^2-\left[\left(r_{t+1}-\hat{r}_{t+1}\right)^2-\left(\bar{r}_{t+1}-\hat{r}_{t+1}\right)^2\right]
\end{equation}

We regress this statistics against a constant and get the one-side p-value.

In [17]:
def get_p_value_of_MSPE_adjusted_test(y:np.ndarray, y_bar:np.ndarray, y_hat:np.ndarray) -> float:
    """
    
    Parameters
    ----------
    y : np.ndarray (n_samples, 1)
    y_bar : np.ndarray (n_samples, 1)
    y_hat : np.ndarray (n_samples, 1)

    Returns
    -------
    p_value_of_MSPE_adjusted : float
    """
    F = (y - y_bar) ** 2 - ((y - y_hat) ** 2 - (y_bar - y_hat) ** 2)
    dummy = np.ones_like(F)
    lm_result = sm.OLS(F, dummy).fit()
    p_value = lm_result.pvalues.values[0]

    return p_value


In [25]:
p_value = prediction_df.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_premium, y_hat=x, y_bar=historical_average))
p_value

Elastic Net      0.103108
Random Forest    0.740081
dtype: float64

In [16]:
def get_significance_of_MSPE_adjusted_test(y:np.ndarray, y_bar:np.ndarray, y_hat:np.ndarray) -> str:
    """
    
    Parameters
    ----------
    y : np.ndarray (n_samples, 1)
    y_bar : np.ndarray (n_samples, 1)
    y_hat : np.ndarray (n_samples, 1)

    Returns
    -------
    significance_of_MSPE_adjusted : str
    """

    p_value = get_p_value_of_MSPE_adjusted_test(y=y, y_hat=y_hat, y_bar=y_bar)
    p_value = round(p_value, ndigits=3)
    if p_value >= 0.1:
        significance = str(p_value) + ' '
    elif p_value > 0.05:
        significance = str(p_value) +' *'
    elif p_value > 0.01:
        significance = str(p_value) +' **'
    elif p_value <= 0.01:
        significance = str(p_value) +' ***'

    return significance

In [26]:
significance = prediction_df.apply(lambda x: get_significance_of_MSPE_adjusted_test(y=equity_premium, y_hat=x, y_bar=historical_average))
significance

Elastic Net      0.103 
Random Forest     0.74 
dtype: object

## The Utility Gain

We assume a mean-variance investor. He balance his portfolio between stock and risk-free bill monthly. The portfolio weights are decided by the prediction of the equity premium. The portfolio weights are given as

\begin{equation}
w_{j, t}=\left(\frac{1}{\gamma}\right)\left(\frac{\hat{r}_{t+1}}{\hat{\sigma}_{t+1}^2}\right)
\end{equation}

This weight is based on the prediction of the stock return and the variance of stock. We use ten years of rolling window for the estimation. This is in line with (Rapach 2010) and Campbell and Thompson (2008). The investor will gain an average utility over out-of-sample period as

\begin{equation}
\hat{v}_0=\hat{\mu}_0-\left(\frac{1}{2}\right) \gamma \hat{\sigma}_0^2
\end{equation}

To get the monthly portfolio weight we set $\gamma = 3$

In [7]:
def get_utility_gain_from_prediction(START_DATE: str,
                                     END_DATE: str,
                                     prediction: pd.DataFrame,
                                     historical_average: pd.DataFrame,
                                     equity_return: pd.DataFrame = None,
                                     rolling_window_size: int = 5, # number in year
                                     data_frequency: int = 12, # number of observations per year
                                     gamma: int = 3) -> float:
    """
    Get utility gain from prediction.
    TODO:
    -----
    1. the calculation of utility gain requires the true return of interested.
    2. we need to replace spy equity premium with the true equity return of interested.

    Parameters
    ----------
    START_DATE : str
        Start date of the utility gain curve.
        Format: YYYY-MM
    END_DATE : str
        End date of the utility gain curve.
        Format: YYYY-MM
    rolling_window_size : int, optional
        Rolling window size.
        Default: 10
    data_frequency : int, optional
        Data frequency.
        Default: 12
    gamma : int, optional
        Gamma.
        Default: 3

    Returns
    -------
    utility_gain : float
        Utility gain.
    """

    START_DATE = datetime.strptime(START_DATE, '%Y-%m')
    START_DATE = str(START_DATE.year - rolling_window_size) + '-' + str(START_DATE.month)
    data_frequency_dict = {12: 'monthly', 4: 'quarterly', 1: 'yearly'}
    econ_predictors = get_econ_predictors(START_DATE=START_DATE, END_DATE=END_DATE, data_freq=data_frequency_dict[data_frequency])
    risk_free_bond = econ_predictors['Treasury Bill'] / 100
    if equity_return is None:
        stock_return = econ_predictors['Equity Premium']
    else:
        stock_return = equity_return[START_DATE:END_DATE]
    portfolio_df = pd.concat([stock_return, risk_free_bond], axis=1).dropna()

    sample_varince = portfolio_df.iloc[:, 0].rolling(rolling_window_size * data_frequency - 1).var().dropna()
    varince_estimation = sample_varince.shift(1).dropna()

    stock_weight_0 = (1 / gamma) * (historical_average / varince_estimation)
    stock_weight_0 = stock_weight_0.clip(0, 1.5)
    portfolio_weight_0 = pd.concat([stock_weight_0, 1 - stock_weight_0], axis = 1)
    w_0 = portfolio_weight_0.values.reshape(-1, 1, 2)

    stock_weight_1 = (1 / gamma) * (prediction / varince_estimation)
    stock_weight_1 = stock_weight_1.clip(0, 1.5)
    portfolio_weight_1 = pd.concat([stock_weight_1, 1 - stock_weight_1], axis = 1)
    w_1 = portfolio_weight_1.values.reshape(-1, 1, 2)

    return_df = portfolio_df.loc[portfolio_weight_0.index] # need to change
    returns = return_df.values.reshape(-1, 2, 1)

    portfolio_return_0 = (w_0 @ returns).flatten()
    portfolio_return_1 = (w_1 @ returns).flatten()

    # annualize the return according to Rapach (2010)
    mu_0 = np.mean(portfolio_return_0) * data_frequency
    sigma_0 = np.var(portfolio_return_0) * data_frequency
    uitility_0 = mu_0 - 0.5 * gamma * sigma_0

    mu_1 = np.mean(portfolio_return_1) * data_frequency
    sigma_1 = np.var(portfolio_return_1) * data_frequency
    uitility_1 = mu_1 - 0.5 * gamma * sigma_1

    utility_gain = uitility_1 - uitility_0
    utility_gain_percentage = utility_gain * 100

    return utility_gain_percentage

In [27]:
utility_gain = prediction_df.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_df.index[0].strftime('%Y-%m'), 
                                                                                END_DATE=prediction_df.index[-1].strftime('%Y-%m'),
                                                                                historical_average=historical_average, 
                                                                                prediction=x))
utility_gain

Elastic Net      0.212202
Random Forest    0.047908
dtype: float64

In [28]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
Elastic Net,0.078502,0.103,0.212202
Random Forest,-10.215775,0.74,0.047908


In [29]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='equity premium out-of-sample ml forecasting performance')

Save table to:../../table/


# Analysis of linear prediction

load prediction results from linear regression (combination)

In [3]:
START_DATE = '1965-01'
END_DATE = '2005-04'
prediction_df = pd.read_csv('../../data/linear_prediction_AG_equity_1947_2005_quarterly.csv', index_col=0, parse_dates=True, date_parser=get_monthly_date_format)
prediction_df = prediction_df.loc[START_DATE:END_DATE]
equity_premium = prediction_df.pop('Equity Premium')
historical_average = prediction_df.pop('Historical Average')

1. R2

In [4]:
R_2_OOS = prediction_df.apply(lambda x: get_oos_r_square(y=equity_premium, y_hat=x, y_bar=historical_average))
R_2_OOS

Dividend Price Ratio     -0.380599
Dividend Yield           -0.443628
Earnings Price Ratio     -0.367326
Earnings Payout Ratio    -1.472913
Stock Variance          -10.576979
Book To Market           -2.552643
Net Equity Expansion     -0.900847
Treasury Bill            -1.610106
Long Term Yield          -2.415471
Long Term Return          0.602016
Term Spread              -2.401840
Default Yield Spread     -2.735281
Default Return Spread    -1.041505
Inflation                 0.745198
Mean                      3.020009
Median                    2.802575
Trimmed mean              2.939501
DMSPE theta 1             3.219785
DMSPE theta 0.9           4.295913
dtype: float64

In [ ]:
R_2_OOS.to_csv('../../data/linear_prediction_quarterly_R2_CL.csv')

2. significance of R2

In [5]:
significance = prediction_df.apply(lambda x: get_significance_of_MSPE_adjusted_test(y=equity_premium, y_hat=x, y_bar=historical_average))
significance

Dividend Price Ratio         0.32 
Dividend Yield              0.279 
Earnings Price Ratio        0.672 
Earnings Payout Ratio       0.653 
Stock Variance            0.042 **
Book To Market              0.759 
Net Equity Expansion         0.61 
Treasury Bill              0.076 *
Long Term Yield             0.279 
Long Term Return            0.217 
Term Spread                0.067 *
Default Yield Spread        0.835 
Default Return Spread       0.519 
Inflation                   0.301 
Mean                     0.005 ***
Median                     0.0 ***
Trimmed mean             0.004 ***
DMSPE theta 1            0.005 ***
DMSPE theta 0.9          0.001 ***
dtype: object

The utility gain

In [8]:
utility_gain = prediction_df.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_df.index[0].strftime('%Y-%m'), 
                                                                                END_DATE=prediction_df.index[-1].strftime('%Y-%m'),
                                                                                historical_average=historical_average, 
                                                                                prediction=x,
                                                                                data_frequency=4))
utility_gain

Dividend Price Ratio     0.171461
Dividend Yield           0.915673
Earnings Price Ratio    -0.616940
Earnings Payout Ratio   -0.246982
Stock Variance          -0.218425
Book To Market          -2.414724
Net Equity Expansion     0.028875
Treasury Bill            3.132849
Long Term Yield          3.048614
Long Term Return         1.338872
Term Spread              3.056866
Default Yield Spread    -0.672800
Default Return Spread    0.534705
Inflation                2.531710
Mean                     3.311734
Median                   2.542841
Trimmed mean             3.121503
DMSPE theta 1            3.417020
DMSPE theta 0.9          4.126452
dtype: float64

In [9]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
Dividend Price Ratio,-0.380599,0.32,0.171461
Dividend Yield,-0.443628,0.279,0.915673
Earnings Price Ratio,-0.367326,0.672,-0.616940
Earnings Payout Ratio,-1.472913,0.653,-0.246982
Stock Variance,-10.576979,0.042 **,-0.218425
Book To Market,-2.552643,0.759,-2.414724
Net Equity Expansion,-0.900847,0.61,0.028875
Treasury Bill,-1.610106,0.076 *,3.132849
Long Term Yield,-2.415471,0.279,3.048614
Long Term Return,0.602016,0.217,1.338872


In [37]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='equity premium out-of-sample linear forecasting performance')

Save table to:../../table/


# Analysis of ML prediction

load ml prediction results

In [63]:
prediction_df = pd.read_csv('../../data/prediction_of_equity_return_ivv_ief_ml.csv', 
                            index_col=0,
                            parse_dates=True,
                            date_parser=get_monthly_date_format)
IVV_IEF_daily_ret = pd.read_csv('../../data/IVV_IEF_daily_ret.csv', index_col=0, parse_dates=True)
IVV_IEF_monthly_ret = IVV_IEF_daily_ret.groupby(pd.Grouper(freq='M')).apply(get_period_return)
IVV_IEF_monthly_ret.index = [get_monthly_date_format(x) for x in IVV_IEF_monthly_ret.index]
equity_return_total_ivv = IVV_IEF_monthly_ret['IVV']
equity_return_total_ief = IVV_IEF_monthly_ret['IEF']
prediction_ivv = prediction_df[['ivv_elastic', 'ivv_rf']]
prediction_ief = prediction_df[['ief_elastic', 'ief_rf']]
historical_average_ivv = prediction_df['ivv_average']
historical_average_ief = prediction_df['ief_average']
equity_return_ivv = prediction_df['ivv_return']
equity_return_ief = prediction_df['ief_return']

R2

In [56]:
R_2_OOS_ief = prediction_ief.apply(lambda x: get_oos_r_square(y=prediction_df['ief_return'], y_hat=x, y_bar=historical_average_ief))
R_2_OOS_ivv = prediction_ivv.apply(lambda x: get_oos_r_square(y=prediction_df['ivv_return'], y_hat=x, y_bar=historical_average_ivv))
R_2_OOS_ivv.index = ['Elastic Net', 'Random Forest']
R_2_OOS_ief.index = ['Elastic Net', 'Random Forest']
R_2_OOS = pd.concat([R_2_OOS_ivv, R_2_OOS_ief], axis=1)
R_2_OOS = R_2_OOS.mean(axis=1)
R_2_OOS.name = 'ML R2'
R_2_OOS

Elastic Net      -5.538171
Random Forest   -19.026376
Name: ML R2, dtype: float64

significance of R2

In [69]:
p_value_ivv = prediction_ivv.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_return_ivv, y_hat=x, y_bar=historical_average_ivv))
p_value_ief = prediction_ief.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_return_ief, y_hat=x, y_bar=historical_average_ief))
p_value_ivv.index = ['Elastic Net', 'Random Forest']
p_value_ief.index = ['Elastic Net', 'Random Forest']
p_value = pd.concat([p_value_ivv, p_value_ief], axis=1)
p_value = p_value.mean(axis=1)
significance = {key:get_significance_of_p_value(value) for key, value in p_value.to_dict().items()}
significance = pd.DataFrame(significance, index=['ML Significance']).T
significance

,ML Significance
Elastic Net,0.847
Random Forest,0.622


The utility gain

In [72]:
utility_gain_ief = prediction_ief.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_ief.index[0].strftime('%Y-%m'), 
                                                                                    END_DATE=prediction_ief.index[-1].strftime('%Y-%m'),
                                                                                    historical_average=historical_average_ief, 
                                                                                    prediction=x,
                                                                                    equity_return=equity_return_total_ief))
utility_gain_ivv = prediction_ivv.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_ivv.index[0].strftime('%Y-%m'), 
                                                                                    END_DATE=prediction_ivv.index[-1].strftime('%Y-%m'),
                                                                                    historical_average=historical_average_ivv, 
                                                                                    prediction=x,
                                                                                    equity_return=equity_return_total_ivv))
utility_gain_ivv.index = ['Elastic Net', 'Random Forest']
utility_gain_ief.index = ['Elastic Net', 'Random Forest']
utility_gain = pd.concat([utility_gain_ivv, utility_gain_ief], axis=1)
utility_gain = utility_gain.mean(axis=1)
utility_gain.name = 'ML Utility Gain'
utility_gain

Elastic Net      0.013332
Random Forest   -0.102459
Name: ML Utility Gain, dtype: float64

In [73]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
Elastic Net,-5.538171,0.847,0.013332
Random Forest,-19.026376,0.622,-0.102459


In [74]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='prediction performance of ML predicting ivv and ief return')

Save table to:../../table/


# Analysis of implied expected return

load implied expected return

In [25]:
prediction_df = pd.read_csv('../../data/implied_equity_premium_1947_2005.csv', index_col=0, parse_dates=True)
prediction_df.index = [get_monthly_date_format(x) for x in prediction_df.index]
prediction_df.columns = ['Equal']

START_DATE = '1965-01'
END_DATE = '2005-04'
prediction_df = prediction_df[START_DATE:END_DATE]

R2

In [26]:
R_2_OOS = prediction_df.apply(lambda x: get_oos_r_square(y=equity_premium, y_hat=x, y_bar=historical_average))
R_2_OOS

Equal    0.664116
dtype: float64

Significance of R2

In [28]:
significance = prediction_df.apply(lambda x: get_significance_of_MSPE_adjusted_test(y=equity_premium, y_hat=x, y_bar=historical_average))
significance

Equal    0.025 **
dtype: object

The utility gain

In [30]:
utility_gain = prediction_df.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_df.index[0].strftime('%Y-%m'), 
                                                                                END_DATE=prediction_df.index[-1].strftime('%Y-%m'),
                                                                                historical_average=historical_average, 
                                                                                prediction=x))
utility_gain

Equal    1.654718
dtype: float64

Output results

In [31]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
Equal,0.664116,0.025 **,1.654718


In [32]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='prediction performance of implied expected return')

Save table to:../../table/
